# **data wrangling**

In [ ]:
#the process of converting data from the initial format to a format that may be better for analysis

In [ ]:
import piplite
await piplite.install(['pandas'])
await piplite.install(['matplotlib'])

In [ ]:
#If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:
#install specific version of libraries used in lab
#! mamba install pandas==1.3.3
#! mamba install numpy=1.21.2

In [6]:
import pandas as pd
import matplotlib.pylab as plt

In [ ]:
#This function will download the dataset into your browser 
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

# **Reading the dataset from the URL and adding the related headers**

In [12]:
filename = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/auto.csv"

In [13]:
headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]

In [ ]:
await download(filename, "auto.csv")
filename="auto.csv"

In [ ]:
df = pd.read_csv(filename, names = headers)

In [ ]:
# To see what the data set looks like, we'll use the head() method.
df.head()

# **Identify and handle missing values**

In [ ]:
#Convert "?" to NaN

import numpy as np

# replace "?" to NaN
df.replace("?", np.nan, inplace = True)
df.head(5)

In [ ]:
missing_data = df.isnull()
missing_data.head(5)

# Count missing values in each column

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

# Deal with missing data:
    # Drop data
    # a. Drop the whole row
    # b. Drop the whole column
    # Replace data
    # a. Replace it by mean
    # b. Replace it by frequency
    # c. Replace it based on other functions

# **Calculate the mean value for the columns, and Replace "NaN" with mean value**

In [ ]:
avg_norm_loss = df["normalized-losses"].astype("float").mean(axis=0)
print("Average of normalized-losses:", avg_norm_loss)
avg_bore=df['bore'].astype('float').mean(axis=0)
print("Average of bore:", avg_bore)
avg_horsepower = df['horsepower'].astype('float').mean(axis=0)
print("Average horsepower:", avg_horsepower)
avg_peakrpm=df['peak-rpm'].astype('float').mean(axis=0)
print("Average peak rpm:", avg_peakrpm)

df["normalized-losses"].replace(np.nan, avg_norm_loss, inplace=True)
df["bore"].replace(np.nan, avg_bore, inplace=True)
df['horsepower'].replace(np.nan, avg_horsepower, inplace=True)
df['peak-rpm'].replace(np.nan, avg_peakrpm, inplace=True)

# **Calculate the frequency value for the columns, and Replace "NaN" with frequency value**

In [ ]:
#To see which values are present in a particular column, we can use the ".value_counts()" method
df['num-of-doors'].value_counts()

# four    114
# two      89

#We can also use the ".idxmax()" method to calculate the most common type automatically
df['num-of-doors'].value_counts().idxmax()

# 'four' 

#replace the missing 'num-of-doors' values by the most frequent 
df["num-of-doors"].replace(np.nan, "four", inplace=True)

# **Drop rows which inlcude NaN**

In [ ]:
# simply drop whole row with NaN in "price" column
df.dropna(subset=["price"], axis=0, inplace=True)

# reset index, because we droped two rows
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

# **Correct data format**

In [ ]:
#.dtype() to check the data type
#.astype() to change the data type

df[["bore", "stroke"]] = df[["bore", "stroke"]].astype("float")
df[["normalized-losses"]] = df[["normalized-losses"]].astype("int")
df[["price"]] = df[["price"]].astype("float")
df[["peak-rpm"]] = df[["peak-rpm"]].astype("float")

df.dtypes

# **Data Standardization**

In [ ]:
#Standardization is the process of transforming data into a common format, allowing the researcher to make 
#the meaningful comparison

# Convert mpg to L/100km by mathematical operation (235 divided by mpg)
df['city-L/100km'] = 235/df["city-mpg"]

# check your transformed data 
df.head()


# transform mpg to L/100km by mathematical operation (235 divided by mpg)
df["highway-mpg"] = 235/df["highway-mpg"]

# rename column name from "highway-mpg" to "highway-L/100km"
df.rename(columns={'highway-mpg':'highway-L/100km'}, inplace=True)

# check your transformed data 
df.head()

# **Data Normalization**

In [ ]:
#Normalization is the process of transforming values of several variables into a similar range. Typical normalizations 
#include scaling the variable so the variable average is 0, scaling the variable so the variance is 1, or scaling the 
#variable so the variable values range from 0 to 1.


# replace (original value) by (original value)/(maximum value)
df['length'] = df['length']/df['length'].max()
df['width'] = df['width']/df['width'].max()

# show the scaled columns
df[["length","width","height"]].head()

# **Binning**

In [ ]:
#inning is a process of transforming continuous numerical variables into discrete categorical 'bins' for grouped analysis

df["horsepower"]=df["horsepower"].astype(int, copy=True)

%matplotlib inline
import matplotlib as plt
from matplotlib import pyplot
plt.pyplot.hist(df["horsepower"])

# set x/y labels and plot title
plt.pyplot.xlabel("horsepower")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower bins")

In [ ]:
bins = np.linspace(min(df["horsepower"]), max(df["horsepower"]), 4)   #4: numbers_generated
bins

In [ ]:
group_names = ['Low', 'Medium', 'High']

In [ ]:
df['horsepower-binned'] = pd.cut(df['horsepower'], bins, labels=group_names, include_lowest=True )
df[['horsepower','horsepower-binned']].head(20)

In [ ]:
df["horsepower-binned"].value_counts()

# Low       153
# Medium     43
# High        5

In [ ]:
%matplotlib inline
import matplotlib as plt
from matplotlib import pyplot
pyplot.bar(group_names, df["horsepower-binned"].value_counts())

# set x/y labels and plot title
plt.pyplot.xlabel("horsepower")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower bins")

In [ ]:
%matplotlib inline
import matplotlib as plt
from matplotlib import pyplot


# draw historgram of attribute "horsepower" with bins = 3
plt.pyplot.hist(df["horsepower"], bins = 3)

# set x/y labels and plot title
plt.pyplot.xlabel("horsepower")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower bins")

# **Indicator Variable (or Dummy Variable)**

In [ ]:
#An indicator variable (or dummy variable) is a numerical variable used to label categories. 
#They are called 'dummies' because the numbers themselves don't have inherent meaning.

df.columns
dummy_variable_1 = pd.get_dummies(df["fuel-type"])
dummy_variable_1.head()

# diesel	gas
# 0	0	1
# 1	0	1
# 2	0	1
# 3	0	1
# 4	0	1

dummy_variable_1.rename(columns={'gas':'fuel-type-gas', 'diesel':'fuel-type-diesel'}, inplace=True)
dummy_variable_1.head()

In [ ]:
# merge data frame "df" and "dummy_variable_1" 
df = pd.concat([df, dummy_variable_1], axis=1)

# drop original column "fuel-type" from "df"
df.drop("fuel-type", axis = 1, inplace=True)

df.head()

In [ ]:
# get indicator variables of aspiration and assign it to data frame "dummy_variable_2"
dummy_variable_2 = pd.get_dummies(df['aspiration'])

# change column names for clarity
dummy_variable_2.rename(columns={'std':'aspiration-std', 'turbo': 'aspiration-turbo'}, inplace=True)

# show first 5 instances of data frame "dummy_variable_1"
dummy_variable_2.head()

# merge the new dataframe to the original datafram
df = pd.concat([df, dummy_variable_2], axis=1)

# drop original column "aspiration" from "df"
df.drop('aspiration', axis = 1, inplace=True)

df.to_csv('clean_df.csv')